# Ejemplo de código de búsqueda con Azure Cognitive Search
Este código muestra cómo usar Azure Cognitive Search con OpenAI y el SDK de Azure Python.

## 1.- Setup inicial

### 1.1- Importacion de librerias

In [33]:
import openai
import os
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector

### 1.2.- Cargar variables de entorno

In [34]:
# Cargar secretos y configuración desde el archivo .env
load_dotenv()

# OpenAI API
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))
print("OpenAI API base: {}".format(openai.api_base))
print("OpenAI API version: {}".format(openai.api_version))
print("OpenAI API type: {}".format(openai.api_type))

# Azure Search API
search_service_name = os.getenv("SEARCH_SERVICE_NAME")
search_service_key = os.getenv("SEARCH_SERVICE_KEY")
search_index_name = os.getenv("SEARCH_INDEX_NAME")
search_endpoint = "https://{}.search.windows.net/".format(search_service_name)
search_vector_config_name = os.getenv("SEARCH_VECTOR_CONFIG_NAME")
search_semantic_config_name = os.getenv("SEARCH_SEMANTIC_CONFIG_NAME")
print("Azure Search service name: {}".format(search_service_name))
print("Azure Search service key: {}".format(search_service_key[:5] + '...' + search_service_key[-5:]))
print("Azure Search index name: {}".format(search_index_name))
print("Azure Search endpoint: {}".format(search_endpoint))
print("Azure Search vector config name: {}".format(search_vector_config_name))
print("Azure Search semantic config name: {}".format(search_semantic_config_name))

OpenAI API key: 1c7f2...3613f
OpenAI API base: https://wsl-openai-canada.openai.azure.com/
OpenAI API version: 2023-03-15-preview
OpenAI API type: azure
Azure Search service name: wsl-cog-search-test-2
Azure Search service key: 9GUM9...z1hrb
Azure Search index name: cogsrch-index
Azure Search endpoint: https://wsl-cog-search-test-2.search.windows.net/
Azure Search vector config name: cogsrch-vector-config
Azure Search semantic config name: cogsrch-semantic-config


### 1.3.- Clase para creación de Clientes para Azure Search

In [20]:
class CreateClient(object):
    def __init__(self, endpoint, key, index_name):
        self.endpoint = endpoint
        self.index_name = index_name
        self.key = key
        self.credentials = AzureKeyCredential(key)

    # Crear un cliente de búsqueda
    # Use esto para cargar documentos al índice
    def create_search_client(self):
        return SearchClient(
            endpoint=self.endpoint,
            index_name=self.index_name,
            credential=self.credentials,
        )

    # Crear un cliente SearchIndex
    # Esto se utiliza para crear, administrar y eliminar un índice.
    def create_admin_client(self):
        return SearchIndexClient(endpoint=self.endpoint, credential=self.credentials)

### Creación de Clientes de Búsqueda de Cognitive Search

In [21]:
base_client = CreateClient(search_endpoint, search_service_key, search_index_name)
search_client = base_client.create_search_client()
admin_client = base_client.create_admin_client()

### Metodo para generación de embeddings

In [22]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(5))
def generate_embeddings(text):
    response = openai.Embedding.create(input=text, engine=embedding_model)
    embeddings = response["data"][0]['embedding']
    return embeddings

## 2.- Busqueda simple

In [23]:
results =  search_client.search(
    query_type='simple',
    query_language='es-es',
    search_text="Donde estan los indicadores de rendimiento SSCC de mayo 2020?" ,
    select=[],
    include_total_count=True
    )

for result in results:
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']}")

Page 3 of DE03599-23.pdf -> 5.3951507
Page 2 of DE03599-23.pdf -> 4.3716464
Page 1 of DE03612-23.pdf -> 3.9833078
Page 1 of DE03599-23.pdf -> 3.9242792
Page 1 of DE03596-23.pdf -> 3.3593283
Page 2 of DE03596-23.pdf -> 3.071132
Page 4 of DE03599-23.pdf -> 2.7912374
Page 2 of DE03598-23.pdf -> 1.678012
Page 1 of DE03591-23.pdf -> 1.604201
Page 3 of DE03596-23.pdf -> 1.4852228
Page 1 of DE03592-23.pdf -> 1.2158293
Page 2 of DE03612-23.pdf -> 1.078795
Page 1 of DE03610-23.pdf -> 1.048455
Page 1 of DE03597-23.pdf -> 1.0325106
Page 1 of DE03590-23.pdf -> 1.0295905
Page 1 of DE03587-23.pdf -> 1.0201067
Page 2 of DE03597-23.pdf -> 0.9028444
Page 1 of DE03598-23.pdf -> 0.57918334


## 3.- Busqueda semantica

In [37]:
results =  search_client.search(
    query_type='semantic', 
    query_language='es-es', 
    semantic_configuration_name=search_semantic_config_name,
    search_text="Donde estan los indicadores de rendimiento SSCC de mayo 2020?" ,
    select=[], 
    query_caption='extractive'
    )

for result in results:
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']} - {result['@search.reranker_score']}")
    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Page 1 of DE03599-23.pdf -> 3.9242792 - 2.8970587253570557
Caption: los indicadores mencionados pueden ser descargados desde la siguiente ruta:    > inicio > operación > servicios complementarios > índices de desempeño y  disponibilidad de servicios complementarios > 2020 ><em> mayo 2020</em> > versión 4     cabe destacar que los indicadores de  los meses de enero  a abril y junio del año 2020, no  sufren modificaciones, …

Page 3 of DE03599-23.pdf -> 5.3951507 - 2.6875765323638916
Caption: Se acoge observación   Esta observación fue recibida en el proceso de  observaciones a los Indicadores de  Desempeño de SSCC de mayo 2020, siendo  incorporada en el cálculo de los indicadores de  mayo 2020 en su versión 3 y contestada en la  carta DE05541- 22 del 5 de diciembre de 2022.

Page 1 of DE03596-23.pdf -> 3.3593283 - 1.592294692993164
Caption: los indicadores  mencionados pueden ser descargados desde la siguiente ruta:    > inicio > operación > servicios complementarios > índices de desemp

## 4.- Busqueda vectorial

In [27]:
user_query = "Donde estan los indicadores de rendimiento SSCC de mayo 2020?"
# Se genera el objeto vectorial de la consulta del usuario con el campo embeddings
vector = Vector(value=generate_embeddings(user_query), k=5, fields="embeddings")

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=[]
    )  
    
# Muestra los resultados de la búsqueda
for result in results: 
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']}")

Page 1 of DE03612-23.pdf -> 0.854698
Page 1 of DE03599-23.pdf -> 0.854264
Page 3 of DE03599-23.pdf -> 0.85401237
Page 1 of DE03596-23.pdf -> 0.8506246
Page 2 of DE03596-23.pdf -> 0.8455717


## 5.- Busqueda hibrida

### 5.1.- Simple + Vector

In [32]:
results = search_client.search(
    query_type='simple',  
    search_text=user_query,  
    vectors=[vector],
    select=[]
)  

# Muestra los resultados de la búsqueda
for result in results:  
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']}")


Page 1 of DE03612-23.pdf -> 0.03279569745063782
Page 3 of DE03599-23.pdf -> 0.03279569745063782
Page 1 of DE03599-23.pdf -> 0.03226646035909653
Page 1 of DE03596-23.pdf -> 0.0314980149269104
Page 2 of DE03596-23.pdf -> 0.03100961446762085
Page 2 of DE03599-23.pdf -> 0.016393441706895828
Page 4 of DE03599-23.pdf -> 0.01515151560306549
Page 2 of DE03598-23.pdf -> 0.014925372786819935
Page 1 of DE03591-23.pdf -> 0.014705882407724857
Page 3 of DE03596-23.pdf -> 0.014492753893136978
Page 1 of DE03592-23.pdf -> 0.014285714365541935
Page 2 of DE03612-23.pdf -> 0.014084506779909134
Page 1 of DE03610-23.pdf -> 0.013888888992369175
Page 1 of DE03597-23.pdf -> 0.013698630034923553
Page 1 of DE03590-23.pdf -> 0.013513513840734959
Page 1 of DE03587-23.pdf -> 0.013333333656191826
Page 2 of DE03597-23.pdf -> 0.01315789483487606
Page 1 of DE03598-23.pdf -> 0.012987012974917889


### 5.2.- Semantic + Vector

In [36]:
results = search_client.search(  
    query_type='semantic',
    query_language='es-es', 
    semantic_configuration_name=search_semantic_config_name,
    query_caption='extractive',            
    search_text=user_query,  
    vectors=[vector],
    select=[]
)  

# Muestra los resultados de la búsqueda
for result in results:
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']} - {result['@search.reranker_score']}")
    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

Page 1 of DE03599-23.pdf -> 0.03226646035909653 - 2.8970587253570557
Caption: los indicadores mencionados pueden ser descargados desde la siguiente ruta:    > inicio > operación > servicios complementarios > índices de desempeño y  disponibilidad de servicios complementarios > 2020 ><em> mayo 2020</em> > versión 4     cabe destacar que los indicadores de  los meses de enero  a abril y junio del año 2020, no  sufren modificaciones, …

Page 3 of DE03599-23.pdf -> 0.03279569745063782 - 2.6875765323638916
Caption: Se acoge observación   Esta observación fue recibida en el proceso de  observaciones a los Indicadores de  Desempeño de SSCC de mayo 2020, siendo  incorporada en el cálculo de los indicadores de  mayo 2020 en su versión 3 y contestada en la  carta DE05541- 22 del 5 de diciembre de 2022.

Page 1 of DE03596-23.pdf -> 0.0314980149269104 - 1.592294692993164
Caption: los indicadores  mencionados pueden ser descargados desde la siguiente ruta:    > inicio > operación > servicios comple